# Pylon Lottery - Galactic Punk

### Pylon Lottery
The pylon lottery is a mechanism implemented in the [Pylon](https://pylon.money/) protocol, in which users can deposit UST in exchange for lottey tickets. Each 10 UST deposited is rewarded with 1 ticket. The longer the UST remain in the pool, the more tickets the user is rewarded. At the end of the lottey period, a winning ticket is extracted from the ones rewarded to users and the lucky user owning the winning ticket wins the price.

### Galactic Punks
I quote from the pylon [protocol page](https://gateway.pylon.money/nft/galactic-punks): *"Galactic Punks are 10,921 randomly generated NFTs on the Terra blockchain. Attributes metadata will be revealed alongside rarity rankings."*


### Objective
In this notebook we will explore the Terra blockchain data to obtain insights in the participation in the Lottery. In specific, we will try to answer the two following questions:
* How much UST has been deposited into the lottery since it’s inception?
* What has been the average UST deposit amount?

### Data
The data has been kindly provided by [Flipside](https://app.flipsidecrypto.com/velocity) and the query used to retrieve the relevant data can be found [here](https://app.flipsidecrypto.com/velocity/queries/5027ee2e-4de7-41dd-8686-072046b3ce8b).

## Analysis

**Imports**

In [202]:
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.figsize'] = (15.0, 5.0)
pd.set_option('display.max_columns', 120)
pd.options.display.max_colwidth = 10000

In [203]:
pylon_gp_df = pd.read_json('https://api.flipsidecrypto.com/api/v2/queries/5027ee2e-4de7-41dd-8686-072046b3ce8b/data/latest')

In [204]:
pylon_gp_df.head(2)

,BLOCK_TIMESTAMP,MSG_VALUE:COINS,MSG_VALUE:CONTRACT,MSG_VALUE:EXECUTE_MSG,MSG_VALUE:EXECUTE_MSG:SEND:AMOUNT,MSG_VALUE:EXECUTE_MSG:SEND:DENOM,MSG_VALUE:SENDER,TX_ID,TYPE_OP
0,2021-11-06T12:04:10Z,[],"""terra126zhrrpkckjs82elgutz5qmqa4zjndfwk5dr2t""","{\n ""send"": {\n ""amount"": ""600013160"",\n ""contract"": ""terra10jrv8wy6s06mku9t6yawt2yr09wjlqsw0qk0vf"",\n ""msg"": ""eyJkZXBvc2l0Ijp7fX0=""\n }\n}","""600013160""",NaN,"""terra16mvrn4at74pcv5sn5k3znhlp8d03cpplqgh5uh""",3D0EFEEACDF74EB52557D7B33C6E41C21886C33DF3C6508D9F397F300A4DCF62,DEPOSIT
1,2021-11-06T22:17:19Z,[],"""terra126zhrrpkckjs82elgutz5qmqa4zjndfwk5dr2t""","{\n ""send"": {\n ""amount"": ""274006177"",\n ""contract"": ""terra10jrv8wy6s06mku9t6yawt2yr09wjlqsw0qk0vf"",\n ""msg"": ""eyJkZXBvc2l0Ijp7fX0=""\n }\n}","""274006177""",NaN,"""terra1kne4ul6n405w769k2p6swx7x4ep30rv24099r8""",57C8DC3E7B31AC96DD24B761DFE139AC4CD77AEF9B8678E6FD2FD56CF0F34F93,DEPOSIT


**Data cleaning and preparation**

In [205]:
pylon_gp_df.columns = ["block_timestamp","coins","contract","execute_msg","send_amount","send_denom","sender","tx_id","type_op"]
pylon_gp_df=pylon_gp_df.drop("coins", axis=1)
pylon_gp_df.send_amount=pylon_gp_df.send_amount.apply(lambda x: x.replace("\"","")).fillna(0).astype('float64')
pylon_gp_df.sender=pylon_gp_df.sender.apply(lambda x: x.replace("\"",""))
pylon_gp_df.contract=pylon_gp_df.contract.apply(lambda x: x.replace("\"",""))
pylon_gp_df.block_timestamp = pd.to_datetime(pylon_gp_df.block_timestamp)
pylon_gp_df["month_day"] = pylon_gp_df.block_timestamp.apply(lambda t: f"{t.month}-{t.day}")
pylon_gp_df = pylon_gp_df.sort_values(by='block_timestamp')

In [206]:
pylon_gp_df.head(2)

,block_timestamp,contract,execute_msg,send_amount,send_denom,sender,tx_id,type_op,month_day
1840,2021-10-27 13:00:05+00:00,terra126zhrrpkckjs82elgutz5qmqa4zjndfwk5dr2t,"{\n ""send"": {\n ""amount"": ""99997274"",\n ""contract"": ""terra10jrv8wy6s06mku9t6yawt2yr09wjlqsw0qk0vf"",\n ""msg"": ""eyJkZXBvc2l0Ijp7fX0=""\n }\n}",99997274.0,NaN,terra1hzsem376yhguf406dq7qj0a203a6y2527zmkpd,1F4726146EA56E85A89712048235D6146297C3EBFE3D72814E1A68D70554FE54,DEPOSIT,10-27
1846,2021-10-27 13:00:05+00:00,terra126zhrrpkckjs82elgutz5qmqa4zjndfwk5dr2t,"{\n ""send"": {\n ""amount"": ""9999728"",\n ""contract"": ""terra10jrv8wy6s06mku9t6yawt2yr09wjlqsw0qk0vf"",\n ""msg"": ""eyJkZXBvc2l0Ijp7fX0=""\n }\n}",9999728.0,NaN,terra1daktentky0kafchz7pn27ndztdv925xz8hw2dk,EE694C1835CFAD59B558B71A2AA73A352A7BE233A5A8F6F8CCCD0B8DB1141B52,DEPOSIT,10-27


### Question 1: How much UST has been deposited into the lottery since it’s inception?

When interacting with the GP lottery smart contract, there are two operations that can be performed: deposit and withdraw.
In order to calculate the total amount of UST deposited since its inception we have to first sum the amounts of UST deposited and the subtract the sum of all the UST withdrawn.

We start by calculating the total amount deposited

In [210]:
total_deposited = pylon_gp_df[pylon_gp_df.type_op == 'DEPOSIT'].send_amount.sum()/1000000
total_deposited

9181487.540842

The total amount deposited since its inception is: 9,181,487.54 UST

Now, let's calculate the total withdrawn since its inception:

In [211]:
total_withdrawn = pylon_gp_df[pylon_gp_df.type_op == 'WITHDRAW'].send_amount.sum()/1000000
total_withdrawn

5588898.694891

The total amount deposited since its inception is: 5,588,898.69 UST.

Now we can calculate the net deposit into the lottery pool to date.

In [212]:
total_deposited - total_withdrawn

3592588.845951

The net deposit into the lottery to date is therefore: 3,592,588.84 UST

### Question 2: What has been the average UST deposit amount?

As explained earlier, there are two operations possibile. We therefore filter only the deposit operations and calculate the average amount in UST per deposit.

In [216]:
pylon_gp_df[pylon_gp_df.type_op == 'DEPOSIT'].send_amount.mean()/1000000

1977.916316424386

The average amount is therefore 1977 UST